In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import lxml.etree as ET
from scipy.spatial import ConvexHull
from IPython.display import SVG

from pydrake.examples.quadrotor import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from spp.bezier import BezierSPP

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [ ]:
def generate_grid_world(shape, start, goal, seed=None):
    '''
        shape: 2-tuple (rows, cols)
        start: 2-element np array of integer index of start cell
        goal: 2-element np array of integer index of goal cell
        seed: rng seed for np or None
    '''

    if seed is not None:
        np.random.seed(seed)

    # Generate room layout on a grid via a simple growth algorithm.
    # States:
    #  -1: Undetermined
    #   0: Outside
    #   1: Inside
    grid = np.zeros(shape) - 1
    # Make the goal always inside, and the start always outside.
    grid[start[0], start[1]] = 0
    grid[goal[0], goal[1]] = 1

    growth_queue = [goal]
    # For each neighbor of an indoor cell, maybe grow the building
    # into it.
    while len(growth_queue) > 0:
        here = growth_queue.pop(0)
        for dx, dy in zip([-1, 1, 0, 0], [0, 0, -1, 1]):
            target = here + np.array([dx, dy])
            # Bounds check
            if target[0] < 0 or target[1] < 0 or target[0] >= grid.shape[0] or target[1] >= grid.shape[1]:
                continue
            # Already-decided check
            if grid[target[0], target[1]] >= 0:
                continue
            grow = np.random.random() > 0.5
            if grow:
                grid[target[0], target[1]] = 1
                growth_queue.append(target)
            else:
                grid[target[0], target[1]] = 0

    # Now extract edges where we might place walls; either
    # outer wall on indoor/outdoor border, or obstacles
    # inside the building.
    indoor_edges = []
    wall_edges = []
    for i in range(-1, grid.shape[0]):
        for j in range(-1, grid.shape[1]):
            # Look in the +x and +y directions, when they're in bound.
            first = np.array([i, j])
            if i < 0 or j < 0:
                first_state = 0
            else:
                first_state = grid[i, j]
            for dx, dy in zip([1, 0], [0, 1]):
                second = first + np.array([dx, dy])
                if second[0] < 0 or second[1] < 0 or second[0] >= grid.shape[0] or second[1] >= grid.shape[1]:
                    second_state = 0
                else:
                    second_state = grid[second[0], second[1]]
                midpoint = (first + second)/2.
                wall_endpoints = [midpoint + np.array([-dy, dx])/2., midpoint + np.array([dy, -dx])/2.]
                if first_state > 0.5 and np.isclose(first_state, second_state):
                    # Both indoors.
                    indoor_edges.append(wall_endpoints)
                elif first_state > 0.5 and second_state < 0.5:
                    # Indoor-outdoor transition going one way.
                    wall_edges.append(wall_endpoints)
                elif first_state < 0.5 and second_state > 0.5:
                    # Indoor-outdoor transition going other way.
                    wall_edges.append(wall_endpoints[::-1])

    return grid, indoor_edges, wall_edges

def draw_grid_world(grid, start, goal, indoor_edges, outdoor_edges):
    plt.figure(dpi=300).set_size_inches(3, 6)
    # Plot the grid
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid.T, cmap="binary", vmin=0, vmax=1)
    # Plot start and goal
    plt.scatter(np.ones(1)*start[0], np.ones(1)*start[1], s=100, marker="+")
    plt.scatter(np.ones(1)*goal[0], np.ones(1)*goal[1], s=100, marker="*")

    # Draw those walls.
    for e1, e2 in indoor_edges:
        plt.plot([e1[0], e2[0]], [e1[1], e2[1]], linestyle="--", c="red")
    for e1, e2 in wall_edges:
        plt.arrow(e1[0], e1[1], (e2 - e1)[0], (e2 - e1)[1], linestyle="-", color="orange", head_width=0.1)

    plt.xlim([-2, grid.shape[0]])
    plt.ylim([-2, grid.shape[1]])
    
# Compile that into a Drake scene by assembling walls, floor, and ceiling tiles together.
def compile_sdf(output_file, grid, start, goal, indoor_edges, wall_edges, seed=None):
    '''
        Glue together constituent SDFs into one big SDF for the whole scene.
    '''

    if seed is not None:
        np.random.seed(seed)

    # These dict list files + their relative weights of being chosen.
    indoor_options = {
        "models/room_gen/half_wall_horizontal.sdf": 0.5,
        "models/room_gen/half_wall_vertical.sdf": 0.5,
        "models/room_gen/wall_with_center_door.sdf": 0.5,
        "": 1.
    }
    wall_options = {
        "models/room_gen/just_wall.sdf": 1.0,
        "models/room_gen/wall_with_center_door.sdf": 0.25,
        "models/room_gen/wall_with_windows.sdf": 0.25
    }

    root_item = ET.Element('sdf', version="1.5", nsmap={'drake': 'drake.mit.edu'})
    world_item = ET.SubElement(root_item, "world", name="building")
    model_item = ET.SubElement(root_item, "model", name="building")
    def include_static_sdf_at_pose(name, uri, tf):
        include_item = ET.SubElement(model_item, "include")
        name_item = ET.SubElement(include_item, "name")
        name_item.text = name
        uri_item = ET.SubElement(include_item, "uri")
        uri_item.text = uri
        static_item = ET.SubElement(include_item, "static")
        static_item.text = "True"
        pose_item = ET.SubElement(include_item, "pose")
        xyz = tf.translation()
        rpy = RollPitchYaw(tf.rotation()).vector()
        pose_item.text = "%f %f %f %f %f %f" % (
            *xyz, *rpy
        )
        
    regions = [HPolyhedron.MakeBox([-2.5, -2.5, 0.2], [grid.shape[0] * 5 + 7.5, 2.175, 2.8]),
               HPolyhedron.MakeBox([-2.5, -2.5, 0.2], [2.175, grid.shape[1] * 5 + 7.5, 2.8]),
               HPolyhedron.MakeBox([grid.shape[0] * 5 + 2.825, -2.5, 0.2],
                                   [grid.shape[0] * 5 + 7.5, grid.shape[1] * 5 + 7.5, 2.8]),
               HPolyhedron.MakeBox([-2.5, grid.shape[1] * 5 + 2.825, 0.2],
                                   [grid.shape[0] * 5 + 7.5, grid.shape[1] * 5 + 7.5, 2.8])]
    
    if grid[0, 0] < 0.5:
        xy = -5*start
        regions.append(HPolyhedron.MakeBox([xy[0]-2.85, xy[1]-2.85, 0.2], [xy[0]+2.175, xy[1]+2.175, 2.8]))
    if grid[1, 0] < 0.5:
        xy = (np.array([1, 0]) - start)*5
        if grid[0, 0] < 0.5:
            regions.append(HPolyhedron.MakeBox([xy[0]-2.85, xy[1]-2.85, 0.2], [xy[0]+2.85, xy[1]+2.175, 2.8]))
        else:
            regions.append(HPolyhedron.MakeBox([xy[0]-2.175, xy[1]-2.85, 0.2], [xy[0]+2.85, xy[1]+2.175, 2.8]))
    if grid[0, 1] < 0.5:
        xy = (np.array([0, 1]) - start)*5
        if grid[0, 0] < 0.5:
            regions.append(HPolyhedron.MakeBox([xy[0]-2.85, xy[1]-2.85, 0.2], [xy[0]+2.175, xy[1]+2.85, 2.8]))
        else:
            regions.append(HPolyhedron.MakeBox([xy[0]-2.85, xy[1]-2.175, 0.2], [xy[0]+2.175, xy[1]+2.85, 2.8]))
        
    # Populate floor and ceilings.
    for i in range(-1, grid.shape[0] + 1):
        for j in range(-1, grid.shape[1] + 1):
            xy = (np.array([i, j]) - start)*5
            # Floor
            tf = RigidTransform(p=np.r_[xy, 0])
            floor_sdf = "models/room_gen/floor_outdoor.sdf"
            if i >= 0 and j >= 0 and i < grid.shape[0] and j < grid.shape[1] and grid[i, j] > 0.5:
                floor_sdf = "models/room_gen/floor_indoor.sdf"
                regions.append(HPolyhedron.MakeBox([xy[0]-2.175, xy[1]-2.175, 0.2], [xy[0]+2.175, xy[1]+2.175, 2.8]))
            include_static_sdf_at_pose("floor_%05d_%05d" % (i, j), floor_sdf, tf)

            # Ceiling, if we're indoors.
            if i < 0 or j < 0 or i >= grid.shape[0] or j >= grid.shape[1] or grid[i, j] < 0.5:
                continue
            tf = RigidTransform(p=np.r_[xy, 0])
            include_static_sdf_at_pose("ceiling_%05d_%05d" % (i, j), "models/room_gen/ceiling.sdf", tf)

    # Outer edges.
    key_options = list(wall_options.keys())
    probs = np.array(list(wall_options.values()))
    probs = probs / np.sum(probs)
    np.random.shuffle(wall_edges)
    for k, (e1, e2) in enumerate(wall_edges):
        # Force first wall to be a door option, so scene is traversable.
        wall_option = ""
        sdf_key = np.random.choice(key_options, p=probs)
        while (k == 0 and "door" not in sdf_key):
            sdf_key = np.random.choice(key_options, p=probs)
        
        # Take their average for the centerpoint, and infer rotation from the points.
        delta = e2 - e1
        theta = np.arctan2(delta[0], delta[1])
        midpoint = (e1 + e2)/2.

        # Coordinate shift into wall blocks: "rooms" are 5m blocks, and shift
        # so start is at the origin.
        midpoint = (midpoint - start) * 5
        
        if "door" in sdf_key:
            dx = np.abs(0.35 * np.cos(theta) + 0.425 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 0.425 * np.cos(theta))
            lb = np.array([midpoint[0] - dx, midpoint[1] - dy , 0.2])
            ub = np.array([midpoint[0] + dx, midpoint[1] + dy , 1.8])
            regions.append(HPolyhedron.MakeBox(lb, ub))
        elif "window" in sdf_key:
            dx = np.abs(0.35 * np.cos(theta) + 0.55 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 0.55 * np.cos(theta))
            lb = np.array([midpoint[0] - dx + 1.25 * np.sin(theta), midpoint[1] - dy + 1.25 * np.cos(theta), 0.95])
            ub = np.array([midpoint[0] + dx + 1.25 * np.sin(theta), midpoint[1] + dy + 1.25 * np.cos(theta), 2.05])
            regions.append(HPolyhedron.MakeBox(lb, ub))
            
            lb = np.array([midpoint[0] - dx - 1.25 * np.sin(theta), midpoint[1] - dy - 1.25 * np.cos(theta), 0.95])
            ub = np.array([midpoint[0] + dx - 1.25 * np.sin(theta), midpoint[1] + dy - 1.25 * np.cos(theta), 2.05])
            regions.append(HPolyhedron.MakeBox(lb, ub))

        tf = RigidTransform(p=np.r_[midpoint, 0], rpy=RollPitchYaw(0, 0, -theta))
        include_static_sdf_at_pose("outer_wall_%05d" % k, sdf_key, tf)

    # Inner edges.
    key_options = list(indoor_options.keys())
    probs = np.array(list(indoor_options.values()))
    probs = probs / np.sum(probs)
    np.random.shuffle(indoor_edges)
    for k, (e1, e2) in enumerate(indoor_edges):
        # Force first wall to be a door option, so scene is traversable.
        wall_option = ""
        sdf_key = np.random.choice(key_options, p=probs)
        # Take their average for the centerpoint, and infer rotation from the points.
        delta = e2 - e1
        theta = np.arctan2(*delta)
        midpoint = (e1 + e2)/2.

        # Coordinate shift into wall blocks: "rooms" are 5m blocks, and shift
        # so start is at the origin.
        midpoint = (midpoint - start) * 5
        
        if sdf_key == "":
            dx = np.abs(0.35 * np.cos(theta) + 2.175 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 2.175 * np.cos(theta))
            lb = np.array([midpoint[0] - dx, midpoint[1] - dy , 0.2])
            ub = np.array([midpoint[0] + dx, midpoint[1] + dy , 2.8])
            regions.append(HPolyhedron.MakeBox(lb, ub))
            continue
        elif "door" in sdf_key:
            dx = np.abs(0.35 * np.cos(theta) + 0.425 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 0.425 * np.cos(theta))
            lb = np.array([midpoint[0] - dx, midpoint[1] - dy , 0.2])
            ub = np.array([midpoint[0] + dx, midpoint[1] + dy , 1.8])
            regions.append(HPolyhedron.MakeBox(lb, ub))
        elif "horizontal" in sdf_key:
            dx = np.abs(0.35 * np.cos(theta) + 1.02 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 1.05 * np.cos(theta))
            lb = np.array([midpoint[0] - dx + 1.25 * np.sin(theta), midpoint[1] - dy - 1.25 * np.cos(theta), 0.2])
            ub = np.array([midpoint[0] + dx + 1.25 * np.sin(theta), midpoint[1] + dy - 1.25 * np.cos(theta), 2.8])
            regions.append(HPolyhedron.MakeBox(lb, ub))
        elif "vertical" in sdf_key:
            dx = np.abs(0.35 * np.cos(theta) + 2.175 * np.sin(theta))
            dy = np.abs(0.35 * np.sin(theta) + 2.175 * np.cos(theta))
            lb = np.array([midpoint[0] - dx, midpoint[1] - dy, 1.7])
            ub = np.array([midpoint[0] + dx, midpoint[1] + dy, 2.8])
            regions.append(HPolyhedron.MakeBox(lb, ub))

        tf = RigidTransform(p=np.r_[midpoint, 0], rpy=RollPitchYaw(0, 0, theta))
        include_static_sdf_at_pose("inner_wall_%05d" % k, sdf_key, tf)

    # Start and end indicators
    tf = RigidTransform(p=np.r_[(start-start)*5, 0])
    include_static_sdf_at_pose("start_indicator", "models/room_gen/start.sdf", tf)


    tf = RigidTransform(p=np.r_[(goal-start)*5, 0])
    include_static_sdf_at_pose("goal_indicator", "models/room_gen/target.sdf", tf)

    tree = ET.ElementTree(root_item)
    tree.write(output_file, pretty_print=True)
    print("Wrote to ", output_file)
    
    return regions

In [ ]:
start=np.array([-1, -1])
goal=np.array([1, 1])
grid, indoor_edges, wall_edges = generate_grid_world(shape=(2, 2), start=start, goal=goal, seed=42)
# draw_grid_world(grid, start, goal, indoor_edges, wall_edges)

regions = compile_sdf("models/room_gen/building.sdf", grid, start, goal, indoor_edges, wall_edges, seed=42)

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()

MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)

# if running_as_notebook:
simulator.set_target_realtime_rate(1.0)

# Set the initial conditions
context = simulator.get_mutable_context()

meshcat.Delete()
for ii in range(len(regions)):
    v = VPolytope(regions[ii])
    meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                            ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))

# Simulate
simulator.AdvanceTo(0.1)

In [ ]:
class FlatnessInverter(LeafSystem):
    def __init__(self, traj, t_offset=0):
        LeafSystem.__init__(self)
        self.traj = traj
        self.port = self.DeclareVectorOutputPort("state", 12, self.DoCalcState, {self.time_ticket()})
        self.t_offset = t_offset
        
    def DoCalcState(self, context, output):
        t = context.get_time() + self.t_offset + 1e-4
        
        q = np.squeeze(self.traj.value(t))
        q_dot = np.squeeze(self.traj.EvalDerivative(t))
        q_ddot = np.squeeze(self.traj.EvalDerivative(t, 2))
        
        fz = np.sqrt(q_ddot[0]**2 + q_ddot[1]**2 + (q_ddot[2] + 9.81)**2)
        r = np.arcsin(-q_ddot[1]/fz)
        p = np.arcsin(q_ddot[0]/fz)
        
        output.set_value(np.concatenate((q, [r, p, 0], q_dot, np.zeros(3))))

In [ ]:
def run_planning(start, goal, weights, save_location=None):
    if save_location is not None and not os.path.exists(save_location):
        os.makedirs(save_location)
        
    grid, indoor_edges, wall_edges = generate_grid_world(shape=(2, 2), start=start, goal=goal)
    file_location = "models/room_gen/building.sdf"
    if save_location is not None:
        file_location = save_location + "/building.sdf"
    regions = compile_sdf(file_location, grid, start, goal, indoor_edges, wall_edges)
    
    order = 5
    continuity = 3
    vel_limit = 3 * np.ones(3)
    start_pose = np.r_[(start-start)*5, 1.]
    goal_pose = np.r_[(goal-start)*5., 1.]
    
    start_setup = time.time()
    b_spp = BezierSPP(regions, order, continuity)
    b_spp.addTimeCost(weights["time"])
    b_spp.addPathLengthCost(weights["norm"])
    b_spp.addVelocityLimits(-vel_limit, vel_limit)
    setup_time = time.time() - start_setup
    
    start_relax = time.time()
    b_traj_relax, result_relax, hard_result_relax = b_spp.SolvePath(start_pose, goal_pose, True, False)
    relax_time = time.time() - start_relax
    
    start_mip = time.time()
    b_traj_mip, result_mip, _ = b_spp.SolvePath(start_pose, goal_pose, False, False)
    mip_time = time.time() - start_mip
    
    planning_results = dict()
    planning_results["order"] = order
    planning_results["continuity"] = continuity
    planning_results["velocity_limit"] = vel_limit
    planning_results["start_pose"] = start_pose
    planning_results["goal_pose"] = goal_pose
    
    planning_results["setup_time"] = setup_time
    planning_results["relaxation_time"] = relax_time
    planning_results["mip_time"] = mip_time
    
    planning_results["relaxation_solver_1_result"] = result_relax.get_solution_result()
    planning_results["relaxation_solver_1_time"] = result_relax.get_solver_details().optimizer_time
    planning_results["relaxation_solver_1_cost"] = result_relax.get_optimal_cost()
    
    if hard_result_relax is not None:
        planning_results["relaxation_solver_2_result"] = hard_result_relax.get_solution_result()
        planning_results["relaxation_solver_2_time"] = hard_result_relax.get_solver_details().optimizer_time
        planning_results["relaxation_solver_total_time"] = (result_relax.get_solver_details().optimizer_time
                                                            + hard_result_relax.get_solver_details().optimizer_time)
        planning_results["relaxation_solver_2_cost"] = hard_result_relax.get_optimal_cost()
    else:
        planning_results["relaxation_solver_2_result"] = None
        planning_results["relaxation_solver_2_time"] = -1
        planning_results["relaxation_solver_total_time"] = -1
        planning_results["relaxation_solver_2_cost"] = -1
    
    planning_results["mip_solver_result"] = result_mip.get_solution_result()
    planning_results["mip_solver_time"] = result_mip.get_solver_details().optimizer_time
    planning_results["mip_solver_cost"] = result_mip.get_optimal_cost()
    
    print("Solve times:", relax_time, mip_time, flush=True)
    
#     pprint.pprint(planning_results)
#     print(planning_results["relaxation_solver_1_cost"], planning_results["relaxation_solver_2_cost"],
#           planning_results["mip_solver_cost"])
    
    if save_location is not None:
        print("Saving files to", save_location, flush=True)
        with open(save_location + '/regions.reg', 'wb') as f:
            pickle.dump(regions, f)
        with open(save_location + "/relaxation_traj.pkl", "wb") as f:
            pickle.dump(b_traj_relax, f, pickle.HIGHEST_PROTOCOL)
        with open(save_location + "/mip_traj.pkl", "wb") as f:
            pickle.dump(b_traj_mip, f, pickle.HIGHEST_PROTOCOL)
        with open(save_location + '/plan_results.pkl', 'wb') as f:
            pickle.dump(planning_results, f)

In [ ]:
start=np.array([-1, -1])
goal=np.array([1, 1])
weights = {"time": 1., "norm": 1.}
np.random.seed(1)
for ii in range(2):
    run_planning(start, goal, weights, "data/room_gen/order_5_room_" + str(ii).zfill(3))

In [ ]:
room = 0

save_location = "data/room_gen/order_5_room_" + str(room).zfill(3)
shutil.copy(save_location + "/building.sdf", "models/room_gen/building.sdf")

regions = None
# with open(save_location + "/regions.reg", "rb") as f:
#     regions = pickle.load(f)

with open(save_location + "/relaxation_traj.pkl", "rb") as f:
    b_traj = pickle.load(f)
# with open(save_location + "/mip_traj.pkl", "rb") as f:
#     b_traj = pickle.load(f)


builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()


if b_traj is not None:
    traj_system = builder.AddSystem(FlatnessInverter(b_traj))
    quad = QuadrotorGeometry.AddToBuilder(builder, traj_system.get_output_port(0), scene_graph)

meshcat_cpp = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)

simulator.set_target_realtime_rate(1.0)

# Set the initial conditions
context = simulator.get_mutable_context()

meshcat.Delete()

if regions is not None:
    for ii in range(len(regions)):
        v = VPolytope(regions[ii])
        meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                                ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))


# Simulate
if b_traj is not None:
    end_time = b_traj.end_time()
    meshcat_cpp.StartRecording()
    simulator.AdvanceTo(end_time)
    meshcat_cpp.PublishRecording()
else:
    simulator.AdvanceTo(0.1)